In [25]:
from IPython.display import clear_output

In [42]:
!pip3 install -r requirements.txt
clear_output()

In [14]:
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib
import plotly.express as px
import plotly.graph_objects as go
import statsmodels
import re


In [41]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

import nltk 
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.stem import PorterStemmer

nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
clear_output()

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import classification_report

In [4]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
from joblib import Parallel, delayed

In [5]:
class AdvancedParser:
    """ Parses out rss feeds"""

    def __init__(self, start_date='2022-10-01', end_date='2022-11-01', topic_urls_dict = {}, col_names=[]):
        
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        self.end_date = datetime.strptime(end_date, '%Y-%m-%d')
        self.topic_urls_dict = topic_urls_dict
        self.all_dfs = []
        self.col_names = col_names
        
        if not self.col_names:
        # These are the common columns I've seen in podcast rss feeds
            self.col_names = ['published_parsed',
                                'id',
                                'links',
                                'summary',
                                'title',
                                'summary_detail',
                                'title_detail',
                                'published',
                                'guidislink']

    def _retrieve_rss_feed(self, url, topic=''):
        xml_data = feedparser.parse(url)
        df = pd.DataFrame(xml_data.entries)
        df = self._ensure_columns(df, self.col_names)
        df['topic'] = topic
        self.all_dfs.append(df)

    @staticmethod
    def _ensure_columns(df, col_names):
        return df[col_names]

    def gather_data(self):
        assert len(self.topic_urls_dict)

        for topic, urls in self.topic_urls_dict.items():
            for url in urls:
                self._retrieve_rss_feed(url, topic)

        final_df = pd.concat(self.all_dfs)
        final_df.reset_index(drop=True, inplace=True)
        return final_df

In [ ]:
topic_urls_dict = {'news' : ["http://rss.cnn.com/rss/cnn_topstories.rss",
                                "https://www.huffpost.com/section/front-page/feed?x=1",
                                "https://feeds.simplecast.com/54nAGcIl",
                                "https://feeds.feedburner.com/Monocle24TheGlobalist",
                                "https://www.theguardian.com/news/series/todayinfocus/podcast.xml"],
                'crime' : ["https://feeds.simplecast.com/qm_9xx0g",
                                "https://rss.art19.com/morbid-a-true-crime-podcast",
                                "https://rss.art19.com/erm-mfm",
                                "https://feeds.megaphone.fm/VMP7924981569",
                                "https://www.omnycontent.com/d/playlist/d83f52e4-2455-47f4-982e-ab790120b954/82e70870-d45e-4b4c-8e17-ab8600091b59/e20bfe1d-24c8-4809-b7b0-ab8600091b62/podcast.rss",
                                "https://feeds.megaphone.fm/darknetdiaries",
                                "https://feeds.simplecast.com/GdzgJRQH",
                                "https://feeds.simplecast.com/xl36XBC2",
                                "https://rss.art19.com/dr-death",
                                "https://podcastfeeds.nbcnews.com/HL4TzgYC"],
                'science' : ["https://feeds.simplecast.com/FO6kxYGj",
                                "http://feeds.feedburner.com/radiolab",
                                "https://media.rss.com/fm/feed.xml",
                                "https://omnycontent.com/d/playlist/e73c998e-6e60-432f-8610-ae210140c5b1/6EA152C0-9E3A-45DE-8672-AE2F0056B113/D8936746-9E22-4DBA-B762-AE2F0056B126/podcast.rss",
                                "https://www.thenakedscientists.com/naked_scientists_podcast.xml"],
                'sport' : ["https://media.rss.com/progressivepodcast/feed.xml",
                                "https://media.rss.com/wholeninesports/feed.xml",
                                "https://media.rss.com/moodysportswithdanandzach/feed.xml",
                                "https://media.rss.com/thejosephvorepodcast/feed.xml",
                                "https://media.rss.com/wholeshabang/feed.xml",
                                "https://mcsorleys.barstoolsports.com/feed/pardon-my-take",
                                "https://feeds.megaphone.fm/ESP2298543312",
                                "https://feeds.megaphone.fm/ESP3500611186",
                                "https://feeds.megaphone.fm/ESP3025643506"]}

a_ps = AdvancedParser('2022-10-01', '2022-11-01', topic_urls_dict)

In [6]:
# all_pc_df = a_ps.gather_data()
# This is saving off the data
# all_pc_df.to_pickle('all_podcast_info.pkl')

# Ingesting the pkl file
all_pc_df = pd.read_pickle('all_podcast_info.pkl')

# Getting rid of rows where summary is not populated
all_pc_df.dropna(inplace=True)
all_pc_df.reset_index(inplace=True, drop=True)

In [7]:
#Checking to see if my data is evenly spread
all_pc_df.groupby('topic').count()

,published_parsed,id,links,summary,title,summary_detail,title_detail,published,guidislink
topic,,,,,,,,,
crime,2424,2424,2424,2424,2424,2424,2424,2424,2424
news,2169,2169,2169,2169,2169,2169,2169,2169,2169
science,3357,3357,3357,3357,3357,3357,3357,3357,3357
sport,2553,2553,2553,2553,2553,2553,2553,2553,2553


In [8]:
# Since the amount for each topic is not the same, I want to balance this to not influence the model. 
all_topics_list = []
for topic, count in all_pc_df.topic.value_counts().to_dict().items():
    all_topics_list.append(all_pc_df[all_pc_df['topic'] == topic])

# Grabbing the small amount to use as our sample amount for underfitting
smallest_topic_amount = min(all_pc_df.topic.value_counts().to_dict().values())

# This is where we will underfit our data
temp_dfs = []
for temp_df in all_topics_list:
    temp_dfs.append(temp_df.sample(smallest_topic_amount))

fitted_df = pd.concat(temp_dfs)

In [9]:
#Checking to see if my data is evenly spread
fitted_df.groupby('topic').count()

,published_parsed,id,links,summary,title,summary_detail,title_detail,published,guidislink
topic,,,,,,,,,
crime,2169,2169,2169,2169,2169,2169,2169,2169,2169
news,2169,2169,2169,2169,2169,2169,2169,2169,2169
science,2169,2169,2169,2169,2169,2169,2169,2169,2169
sport,2169,2169,2169,2169,2169,2169,2169,2169,2169


In [16]:
# Keep target (topic) and ntext
cols_to_keep = ['topic', 'summary']
train_df = fitted_df[cols_to_keep].copy()

In [44]:
# This code was found on https://medium.com/p/f2fc1796e8c7
#Lets normalize the data

def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(<.*?>)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df

clean_df = clean_text(train_df, 'summary', 'clean_summary')
clean_df.tail()

,topic,summary,cleam_summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma,summary_tokens_pos_tagged
1699,news,President Barack Obama said not to hire him. T...,president barack obama said not to hire him th...,president barack obama said not to hire him th...,"[president, barack, obama, said, hire, attorne...","[presid, barack, obama, said, hire, attorney, ...","[president, barack, obama, said, hire, attorne...","[(president, NN), (barack, NN), (obama, NN), (..."
1254,news,"After the divisiveness of the 2016 election, t...",after the divisiveness of the election the wo...,after the divisiveness of the election the wo...,"[divisiveness, election, womens, march, became...","[divis, elect, women, march, becam, major, sym...","[divisiveness, election, woman, march, became,...","[(divisiveness, JJ), (election, NN), (womens, ..."
421,news,This episode contains descriptions of violence...,this episode contains descriptions of violence...,this episode contains descriptions of violence...,"[episode, contains, descriptions, violence, su...","[episod, contain, descript, violenc, suicid, a...","[episode, contains, description, violence, sui...","[(episode, NN), (contains, NNS), (descriptions..."
804,news,"For the remainder of this week, “The Daily” is...",for the remainder of this week the daily is re...,for the remainder of this week the daily is re...,"[remainder, week, daily, revisiting, episodes,...","[remaind, week, daili, revisit, episod, peopl,...","[remainder, week, daily, revisiting, episode, ...","[(remainder, NN), (week, NN), (daily, RB), (re..."
360,news,"This week, The Daily is revisiting some of our...",this week the daily is revisiting some of our ...,this week the daily is revisiting some of our ...,"[week, daily, revisiting, favorite, episodes, ...","[week, daili, revisit, favorit, episod, year, ...","[week, daily, revisiting, favorite, episode, y...","[(week, NN), (daily, RB), (revisiting, VBG), (..."


In [45]:
# getting rid of stop words

clean_df['clean_summary'] = clean_df['clean_summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
clean_df.head()

,topic,summary,cleam_summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma,summary_tokens_pos_tagged
7045,science,Since helium-3 can be used to power the fusion...,since helium can be used to power the fusion r...,since helium used power fusion reactors future...,"[since, helium, used, power, fusion, reactors,...","[sinc, helium, use, power, fusion, reactor, fu...","[since, helium, used, power, fusion, reactor, ...","[(since, IN), (helium, NN), (used, VBN), (powe..."
6425,science,As we celebrate Chinese New Year and enter the...,as we celebrate chinese new year and enter the...,celebrate chinese new year enter year fire mon...,"[celebrate, chinese, new, year, enter, year, f...","[celebr, chines, new, year, enter, year, fire,...","[celebrate, chinese, new, year, enter, year, f...","[(celebrate, VB), (chinese, JJ), (new, JJ), (y..."
6149,science,<p>Why does heredity hold such power over us? ...,why does heredity hold such power over us how ...,heredity hold power us ancients contemplate ge...,"[heredity, hold, power, us, ancients, contempl...","[hered, hold, power, us, ancient, contempl, ge...","[heredity, hold, power, u, ancient, contemplat...","[(heredity, NN), (hold, VBP), (power, NN), (us..."
6923,science,The human brain strives to organize and classi...,the human brain strives to organize and classi...,human brain strives organize classify sensory ...,"[human, brain, strives, organize, classify, se...","[human, brain, strive, organ, classifi, sensor...","[human, brain, strives, organize, classify, se...","[(human, JJ), (brain, NN), (strives, NNS), (or..."
6307,science,Patient zero looms large in our collective con...,patient zero looms large in our collective con...,patient zero looms large collective consciousn...,"[patient, zero, looms, large, collective, cons...","[patient, zero, loom, larg, collect, conscious...","[patient, zero, loom, large, collective, consc...","[(patient, NN), (zero, NN), (looms, VBZ), (lar..."


In [46]:
# tokenize the summaries

clean_df['summary_tokens'] = clean_df['clean_summary'].apply(lambda x: word_tokenize(x))
clean_df.head()

,topic,summary,cleam_summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma,summary_tokens_pos_tagged
7045,science,Since helium-3 can be used to power the fusion...,since helium can be used to power the fusion r...,since helium used power fusion reactors future...,"[since, helium, used, power, fusion, reactors,...","[sinc, helium, use, power, fusion, reactor, fu...","[since, helium, used, power, fusion, reactor, ...","[(since, IN), (helium, NN), (used, VBN), (powe..."
6425,science,As we celebrate Chinese New Year and enter the...,as we celebrate chinese new year and enter the...,celebrate chinese new year enter year fire mon...,"[celebrate, chinese, new, year, enter, year, f...","[celebr, chines, new, year, enter, year, fire,...","[celebrate, chinese, new, year, enter, year, f...","[(celebrate, VB), (chinese, JJ), (new, JJ), (y..."
6149,science,<p>Why does heredity hold such power over us? ...,why does heredity hold such power over us how ...,heredity hold power us ancients contemplate ge...,"[heredity, hold, power, us, ancients, contempl...","[hered, hold, power, us, ancient, contempl, ge...","[heredity, hold, power, u, ancient, contemplat...","[(heredity, NN), (hold, VBP), (power, NN), (us..."
6923,science,The human brain strives to organize and classi...,the human brain strives to organize and classi...,human brain strives organize classify sensory ...,"[human, brain, strives, organize, classify, se...","[human, brain, strive, organ, classifi, sensor...","[human, brain, strives, organize, classify, se...","[(human, JJ), (brain, NN), (strives, NNS), (or..."
6307,science,Patient zero looms large in our collective con...,patient zero looms large in our collective con...,patient zero looms large collective consciousn...,"[patient, zero, looms, large, collective, cons...","[patient, zero, loom, larg, collect, conscious...","[patient, zero, loom, large, collective, consc...","[(patient, NN), (zero, NN), (looms, VBZ), (lar..."


In [47]:
# Stemming the summary words

def word_stemmer(text):
    stem_text = [PorterStemmer().stem(i) for i in text]
    return stem_text

clean_df['summary_tokens_stem'] = clean_df['summary_tokens'].apply(lambda x: word_stemmer(x))
clean_df.head()

,topic,summary,cleam_summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma,summary_tokens_pos_tagged
7045,science,Since helium-3 can be used to power the fusion...,since helium can be used to power the fusion r...,since helium used power fusion reactors future...,"[since, helium, used, power, fusion, reactors,...","[sinc, helium, use, power, fusion, reactor, fu...","[since, helium, used, power, fusion, reactor, ...","[(since, IN), (helium, NN), (used, VBN), (powe..."
6425,science,As we celebrate Chinese New Year and enter the...,as we celebrate chinese new year and enter the...,celebrate chinese new year enter year fire mon...,"[celebrate, chinese, new, year, enter, year, f...","[celebr, chines, new, year, enter, year, fire,...","[celebrate, chinese, new, year, enter, year, f...","[(celebrate, VB), (chinese, JJ), (new, JJ), (y..."
6149,science,<p>Why does heredity hold such power over us? ...,why does heredity hold such power over us how ...,heredity hold power us ancients contemplate ge...,"[heredity, hold, power, us, ancients, contempl...","[hered, hold, power, us, ancient, contempl, ge...","[heredity, hold, power, u, ancient, contemplat...","[(heredity, NN), (hold, VBP), (power, NN), (us..."
6923,science,The human brain strives to organize and classi...,the human brain strives to organize and classi...,human brain strives organize classify sensory ...,"[human, brain, strives, organize, classify, se...","[human, brain, strive, organ, classifi, sensor...","[human, brain, strives, organize, classify, se...","[(human, JJ), (brain, NN), (strives, NNS), (or..."
6307,science,Patient zero looms large in our collective con...,patient zero looms large in our collective con...,patient zero looms large collective consciousn...,"[patient, zero, looms, large, collective, cons...","[patient, zero, loom, larg, collect, conscious...","[patient, zero, loom, large, collective, consc...","[(patient, NN), (zero, NN), (looms, VBZ), (lar..."


In [48]:
# Lemmatize the summary words

def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

clean_df['summary_tokens_lemma'] = clean_df['summary_tokens'].apply(lambda x: word_lemmatizer(x))
clean_df.head()

,topic,summary,cleam_summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma,summary_tokens_pos_tagged
7045,science,Since helium-3 can be used to power the fusion...,since helium can be used to power the fusion r...,since helium used power fusion reactors future...,"[since, helium, used, power, fusion, reactors,...","[sinc, helium, use, power, fusion, reactor, fu...","[since, helium, used, power, fusion, reactor, ...","[(since, IN), (helium, NN), (used, VBN), (powe..."
6425,science,As we celebrate Chinese New Year and enter the...,as we celebrate chinese new year and enter the...,celebrate chinese new year enter year fire mon...,"[celebrate, chinese, new, year, enter, year, f...","[celebr, chines, new, year, enter, year, fire,...","[celebrate, chinese, new, year, enter, year, f...","[(celebrate, VB), (chinese, JJ), (new, JJ), (y..."
6149,science,<p>Why does heredity hold such power over us? ...,why does heredity hold such power over us how ...,heredity hold power us ancients contemplate ge...,"[heredity, hold, power, us, ancients, contempl...","[hered, hold, power, us, ancient, contempl, ge...","[heredity, hold, power, u, ancient, contemplat...","[(heredity, NN), (hold, VBP), (power, NN), (us..."
6923,science,The human brain strives to organize and classi...,the human brain strives to organize and classi...,human brain strives organize classify sensory ...,"[human, brain, strives, organize, classify, se...","[human, brain, strive, organ, classifi, sensor...","[human, brain, strives, organize, classify, se...","[(human, JJ), (brain, NN), (strives, NNS), (or..."
6307,science,Patient zero looms large in our collective con...,patient zero looms large in our collective con...,patient zero looms large collective consciousn...,"[patient, zero, looms, large, collective, cons...","[patient, zero, loom, larg, collect, conscious...","[patient, zero, loom, large, collective, consc...","[(patient, NN), (zero, NN), (looms, VBZ), (lar..."


In [49]:
# This is performing Part of Speech tagging
def word_pos_tagger(text):
    pos_tagged_text = nltk.pos_tag(text)
    return pos_tagged_text

clean_df['summary_tokens_pos_tagged'] = clean_df['summary_tokens'].apply(lambda x: word_pos_tagger(x))
clean_df.head()

,topic,summary,cleam_summary,clean_summary,summary_tokens,summary_tokens_stem,summary_tokens_lemma,summary_tokens_pos_tagged
7045,science,Since helium-3 can be used to power the fusion...,since helium can be used to power the fusion r...,since helium used power fusion reactors future...,"[since, helium, used, power, fusion, reactors,...","[sinc, helium, use, power, fusion, reactor, fu...","[since, helium, used, power, fusion, reactor, ...","[(since, IN), (helium, NN), (used, VBN), (powe..."
6425,science,As we celebrate Chinese New Year and enter the...,as we celebrate chinese new year and enter the...,celebrate chinese new year enter year fire mon...,"[celebrate, chinese, new, year, enter, year, f...","[celebr, chines, new, year, enter, year, fire,...","[celebrate, chinese, new, year, enter, year, f...","[(celebrate, VB), (chinese, JJ), (new, JJ), (y..."
6149,science,<p>Why does heredity hold such power over us? ...,why does heredity hold such power over us how ...,heredity hold power us ancients contemplate ge...,"[heredity, hold, power, us, ancients, contempl...","[hered, hold, power, us, ancient, contempl, ge...","[heredity, hold, power, u, ancient, contemplat...","[(heredity, NN), (hold, VBP), (power, NN), (us..."
6923,science,The human brain strives to organize and classi...,the human brain strives to organize and classi...,human brain strives organize classify sensory ...,"[human, brain, strives, organize, classify, se...","[human, brain, strive, organ, classifi, sensor...","[human, brain, strives, organize, classify, se...","[(human, JJ), (brain, NN), (strives, NNS), (or..."
6307,science,Patient zero looms large in our collective con...,patient zero looms large in our collective con...,patient zero looms large collective consciousn...,"[patient, zero, looms, large, collective, cons...","[patient, zero, loom, larg, collect, conscious...","[patient, zero, loom, large, collective, consc...","[(patient, NN), (zero, NN), (looms, VBZ), (lar..."


In [92]:
# also need to map topics to nums (targets)
topics = clean_df.topic.unique().tolist()
topic_map = dict(zip(topics, list(range(0, len(topics)))))
inverse_topic_map = dict(zip(list(range(0, len(topics))), topics))
clean_df['topic_num'] = clean_df['topic'].apply(lambda x: topic_map.get(x))

In [75]:
# Used https://towardsdatascience.com/how-to-enter-your-first-kaggle-competition-4717e7b232db as resource
# Splitting our data to test and train and creating a model
X_train, X_test, y_train, y_test = train_test_split(clean_df['clean_summary'],clean_df['topic_num'],random_state = 0)
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', LogisticRegression()),
])
model = pipeline_sgd.fit(X_train, y_train)

In [76]:
y_predict = model.predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       537
           1       1.00      0.98      0.99       563
           2       0.99      0.97      0.98       543
           3       0.97      0.97      0.97       526

    accuracy                           0.98      2169
   macro avg       0.98      0.98      0.98      2169
weighted avg       0.98      0.98      0.98      2169



In [80]:
# Let's bring in different podcasts to test our model more

test_topic_urls_dict = {
                        'news' : ["https://feeds.megaphone.fm/slatesthegist"],
                        'crime' : ["https://audioboom.com/channels/4940872.rss",
                                "https://rss.art19.com/generation-why-podcast"],
                        'science' : ["https://feed.theskepticsguide.org/feed/rss.aspx?feed=sgu"], 
                        'sport' : ["https://feeds.megaphone.fm/ESP4820632502"]
}
test_parse = AdvancedParser('2022-10-01', '2022-11-01', test_topic_urls_dict)
test_df = test_parse.gather_data()

In [81]:
# map topics to nums (targets)
test_df['topic_num'] = test_df['topic'].apply(lambda x: topic_map.get(x))

In [87]:
submission_test_clean = test_df.copy()
submission_test_clean = clean_text(submission_test_clean, "summary", "summary_clean")
submission_test_clean['summary'] = submission_test_clean['summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
submission_test_clean.head()

,published_parsed,id,links,summary,title,summary_detail,title_detail,published,guidislink,topic,topic_num,summary_clean
0,"(2022, 11, 14, 23, 22, 0, 0, 318, 0)",46e05288-4e2a-11ed-909a-a7bb01bdf4e1,"[{'length': '0', 'type': 'audio/mpeg', 'href':...",Princess Martha Louise Norway engaged American...,The Princess And The Shaman (Who Might Also Be...,"{'type': 'text/plain', 'language': None, 'base...","{'type': 'text/plain', 'language': None, 'base...","Mon, 14 Nov 2022 23:22:00 -0000",False,news,3,princess martha louise of norway is engaged to...
1,"(2022, 11, 12, 22, 30, 0, 5, 316, 0)",45be14ee-4e2a-11ed-909a-9bf573d3c5c0,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","In installment Best Of The Gist, look squishie...",BEST OF THE GIST: Squishy Science Edition,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Sat, 12 Nov 2022 22:30:00 -0000",False,news,3,in this installment of best of the gist we loo...
2,"(2022, 11, 11, 22, 0, 0, 4, 315, 0)",460fe80a-4e2a-11ed-909a-fb0f0740e50b,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","The pundit, case, The Gist host Mike Pesca. In...",What Went Right For The Dems And Wrong For The...,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Fri, 11 Nov 2022 22:00:00 -0000",False,news,3,the pundit in this case being the gist host mi...
3,"(2022, 11, 10, 22, 38, 44, 3, 314, 0)",461f40ac-4e2a-11ed-909a-db5ff2299d5f,"[{'length': '0', 'type': 'audio/mpeg', 'href':...",As wait … wait … wait results states like Ariz...,Hot-Dog And The Arizona Results,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Thu, 10 Nov 2022 22:38:44 -0000",False,news,3,as we just wait wait wait for the results in...
4,"(2022, 11, 9, 23, 17, 0, 2, 313, 0)",466f466a-4e2a-11ed-909a-bfc42273a75b,"[{'length': '0', 'type': 'audio/mpeg', 'href':...","Californians won’t gambling sports, Oregon leg...",A Red Ripple,"{'type': 'text/html', 'language': None, 'base'...","{'type': 'text/plain', 'language': None, 'base...","Wed, 09 Nov 2022 23:17:00 -0000",False,news,3,californians wont be gambling on sports oregon...


In [88]:
# Run the model
submission_test_pred = model.predict(submission_test_clean['summary'])

In [90]:
submission_test_clean['prediction'] = submission_test_pred

In [93]:
submission_test_clean['prediction_topic'] = submission_test_clean['prediction'].apply(lambda x: inverse_topic_map.get(x))

In [100]:
submission_test_clean[['topic', 'prediction_topic']][:20]

,topic,prediction_topic
0,news,science
1,news,sport
2,news,sport
3,news,sport
4,news,sport
5,news,science
6,news,sport
7,news,sport
8,news,sport
9,news,sport
